In [1]:
print("test")

test


In [2]:
import os
from pathlib import Path

current_directory = Path.cwd()

root_directory = current_directory
os.chdir(root_directory)



In [3]:
import argparse
import os

import torch
import tqdm
import yaml
from attrdict import AttrDict

from modules.comp.comp_d_net_pl import *
from modules.mono.depth_net_pl import *
from modules.mv.mv_depth_net_pl import *
from utils.data_utils import *
from utils.localization_utils import *


from torch.utils.data import DataLoader
import lightning as Lightning

In [4]:
net_type = "d"
dataset = "gibson_f"
dataset_path = r"C:\Users\matth\Downloads\Gibson_Floorplan_Localization_Dataset"


In [5]:
# get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("======= USING DEVICE : ", device, " =======")

======= USING DEVICE :  cpu  =======


In [6]:
# parameters
L = 3  # number of the source frames
D = 128  # number of depth planes
d_min = 0.1  # minimum depth
d_max = 15.0  # maximum depth
d_hyp = -0.2  # depth transform (uniform sampling in d**d_hyp)
F_W = 3 / 8  # camera intrinsic, focal length / image width
trans_thresh = 0.005  # translation threshold (variance) if using comp_s

add_rp = (
    False  # whether use roll and pitch angle augmentation, only used in training
)
roll = 0  # maximum roll augmentation in randian
pitch = 0  # maximum pitch augmentation in randian

add_rp = (
    True  # whether use roll and pitch angle augmentation, only used in training
)
roll = 0.3  # maximum roll augmentation in randian
pitch = 0.3  # maximum pitch augmentation in randian

In [7]:
# paths
dataset_dir = os.path.join(dataset_path, dataset)
depth_dir = dataset_dir
#log_dir = ckpt_path
desdf_path = os.path.join(dataset_path, "desdf")

if net_type == "d":
    depth_suffix = "depth40"
else:
    depth_suffix = "depth160"

In [8]:
# instantiate dataset
split_file = os.path.join(dataset_dir, "split.yaml")
with open(split_file, "r") as f:
    split = AttrDict(yaml.safe_load(f))

In [9]:
split.train

('Albertville',
 'Anaheim',
 'Andover',
 'Angiola',
 'Annawan',
 'Applewold',
 'Arkansaw',
 'Avonia',
 'Azusa',
 'Ballou',
 'Beach',
 'Bolton',
 'Bonfield',
 'Brentsville',
 'Brevort',
 'Cantwell',
 'Capistrano',
 'Caruthers',
 'Colebrook',
 'Connellsville',
 'Convoy',
 'Cooperstown',
 'Crandon',
 'Dauberville',
 'Delton',
 'Denmark',
 'Dryville',
 'Dunmor',
 'Eagerville',
 'Eastville',
 'Edgemere',
 'Elmira',
 'Espanola',
 'Eudora',
 'Ewansville',
 'Goffs',
 'Greigsville',
 'Hainesburg',
 'Hambleton',
 'Haxtun',
 'Hillsdale',
 'Hometown',
 'Hominy',
 'Kerrtown',
 'Ladue',
 'Lathrup',
 'Lovilia',
 'Maben',
 'Maryhill',
 'Matoaca',
 'Mesic',
 'Mobridge',
 'Monson',
 'Morris',
 'Mosinee',
 'Mosquito',
 'Nicut',
 'Nimmons',
 'Norvelt',
 'Nuevo',
 'Oyens',
 'Pablo',
 'Parole',
 'Pettigrew',
 'Placida',
 'Pleasant',
 'Plessis',
 'Quantico',
 'Rabbit',
 'Rancocas',
 'Reyno',
 'Ribera',
 'Roane',
 'Rockport',
 'Roeville',
 'Rosenberg',
 'Rosser',
 'Roxboro',
 'Samuels',
 'Sanctuary',
 'Albert

In [14]:
# Define data
dataset = GridSeqDataset(
    dataset_dir,
    split.train,
    L=L,
    depth_dir=depth_dir,
    depth_suffix=depth_suffix,
    add_rp=add_rp,
    roll=roll,
    pitch=pitch,
)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [15]:
next(iter(dataloader))#["ref_depth"]

{'ref_depth': tensor([[0.7177, 0.7383, 0.7602,  ..., 1.6845, 1.6387, 1.5896],
         [3.1817, 3.3139, 3.4575,  ..., 2.3058, 2.2666, 2.2288],
         [1.8948, 1.8342, 1.7824,  ..., 0.8866, 0.8681, 0.8623],
         ...,
         [3.1128, 3.1636, 3.2160,  ..., 2.0678, 2.1527, 2.1205],
         [0.9104, 0.8958, 0.8724,  ..., 0.1909, 0.1885, 0.1863],
         [0.2134, 0.2131, 0.2128,  ..., 0.2023, 0.2020, 0.2017]]),
 'src_depth': tensor([[[0.8080, 0.8381, 0.8705,  ..., 1.9328, 1.8928, 1.8479],
          [0.7722, 0.7986, 0.8268,  ..., 1.8202, 1.7740, 1.7292],
          [0.7427, 0.7661, 0.7909,  ..., 1.7432, 1.6939, 1.6474]],
 
         [[3.7242, 3.7130, 3.6975,  ..., 3.2460, 3.2342, 3.2225],
          [3.5210, 3.6912, 3.8787,  ..., 2.8721, 2.8570, 2.8337],
          [3.3680, 3.5217, 3.6902,  ..., 2.6002, 2.5694, 2.5394]],
 
         [[1.5123, 1.5130, 1.5137,  ..., 1.4690, 1.4707, 1.4645],
          [1.5518, 1.5390, 1.5327,  ..., 1.2325, 1.2309, 1.2212],
          [1.6036, 1.5786, 1.5587,

In [16]:
# Define model 
#model = depth_net_pl(d_min=d_min, d_max=d_max, d_hyp=d_hyp, D=D)
model = comp_d_net_pl(mv_net=mv_depth_net_pl(D=D, d_hyp=d_hyp).net,
            mono_net=depth_net_pl(d_min=d_min, d_max=d_max, d_hyp=d_hyp, D=D).encoder,
            L=L,
            d_min=d_min,
            d_max=d_max,
            d_hyp=d_hyp,
            D=D,
            F_W=F_W,
            use_pred=True).to(device)

In [17]:
# Train the model
trainer = Lightning.Trainer(fast_dev_run=10)
#trainer.fit(model, dataset)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 100 batch(es). Logging and checkpointing is suppressed.



  | Name       | Type       | Params
------------------------------------------
0 | comp_d_net | comp_d_net | 26.4 M
------------------------------------------
5.3 K     Trainable params
26.4 M    Non-trainable params
26.4 M    Total params
105.718   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/100 [00:00<?, ?it/s] 

c:\Users\matth\Documents\ETHZ\01_DS\03_CapstoneProject\04_Code\f3loc\modules\comp\comp_d_net_pl.py:59: UserWarning: Using a target size (torch.Size([64, 40])) that is different to the input size (torch.Size([64, 160])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l1_loss = F.l1_loss(d_comp, batch["ref_depth"])


Epoch 0:   0%|          | 0/100 [26:48<?, ?it/s]


RuntimeError: The size of tensor a (160) must match the size of tensor b (40) at non-singleton dimension 1

In [23]:
len(dataset.gt_pose)

100